In [24]:
# pip install numpy==1.19.5 # numpy 버전 이슈로 LSTM 실행이 안됨

In [25]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
import time

In [26]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(
    num_words = 10000
)

print(x_train.shape, x_test.shape)

(8982,) (2246,)


In [27]:
# print(np.unique(y_train, return_counts=True))   # 데이터 확인
print(x_train.shape, x_test.shape)
print(len(np.unique(y_train)))  # 46

(8982,) (2246,)
46


In [28]:
print('뉴스 기사의 최대 길이 : ', max(len(i) for i in x_train))
print('뉴스 기사의 평균 길이 : ', sum(map(len, x_train)) / len(x_train))

뉴스 기사의 최대 길이 :  2376
뉴스 기사의 평균 길이 :  145.5398574927633


In [29]:
#pad_sequences
x_train = pad_sequences(x_train, padding = 'pre', maxlen = 100)
x_test = pad_sequences(x_test, padding='pre', maxlen = 100)

In [30]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(8982, 100) (8982,)
(2246, 100) (2246,)


In [31]:
# 2. 모델 구성
model = Sequential()
model.add(Embedding(input_dim = 10000,
                   output_dim = 100))  # maxlen = 100
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(46, activation='softmax'))   # 다중분류 이므로 softmax

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)               

In [34]:
# 3. 컴파일, 훈련
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics='accuracy')

## EarlyStopping
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Model Check Point
mcp = ModelCheckpoint(
    monitor='val_loss',
    mode='auto',
    verbose=1,
    # save_weights_only=False,
    save_best_only=True,
    filepath='./_mcp/embedding_reuter.hdf5'
)

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, mode='min',  # mode='auto'가 기본, patience=100 --> 100부터 줄여나가기
                              verbose=1, restore_best_weights=True) # restore_best_weights --> default 는 False 이므로 True 로 꼭!!! 변경!!!  

start_time = time.time()    # 시작 시간

model.fit(x_train, y_train, epochs=50, batch_size=256,
         validation_split=0.2, callbacks=[earlyStopping, mcp])

end_time = time.time() - start_time


Epoch 1/50
29/29 [==============================] - ETA: 0s - loss: 9247157248.0000 - accuracy: 0.3449
Epoch 00001: val_loss improved from inf to 2.35302, saving model to ./_mcp/embedding_reuter.hdf5
29/29 [==============================] - 14s 472ms/step - loss: 9247157248.0000 - accuracy: 0.3449 - val_loss: 2.3530 - val_accuracy: 0.3450
Epoch 2/50
29/29 [==============================] - ETA: 0s - loss: 2.3583 - accuracy: 0.3478
Epoch 00002: val_loss did not improve from 2.35302
29/29 [==============================] - 14s 468ms/step - loss: 2.3583 - accuracy: 0.3478 - val_loss: 9.7856 - val_accuracy: 0.3472
Epoch 3/50
29/29 [==============================] - ETA: 0s - loss: 2060780288.0000 - accuracy: 0.3740
Epoch 00003: val_loss did not improve from 2.35302
29/29 [==============================] - 14s 484ms/step - loss: 2060780288.0000 - accuracy: 0.3740 - val_loss: 3443.6812 - val_accuracy: 0.3795
Epoch 4/50
29/29 [==============================] - ETA: 0s - loss: 931489.7500 - ac

In [33]:
# 4. 평가, 예측
loss, acc = model.evaluate(x_test, y_test)
print('loss : ', loss)
print('acc : ', acc)

71/71 [==============================] - 4s 58ms/step - loss: 2.4042 - accuracy: 0.3620
loss :  2.404205799102783
acc :  0.36197686195373535
